# KBIS 2025
## Kitchen and Bath Industry Show
### February 25-27, 2025
Las Vegas Convention Center 

Contains: Company, booth number, website, city, state, country

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
r = requests.get("https://kbis.a2zinc.net/kbis2025/Public/eventmap.aspx?shMode=E&thumbnail=1&ID=50797")

In [3]:
soup = BeautifulSoup(r.text, "html.parser")

In [4]:
companies = soup.findAll("a", {"class" : "exhibitorName"})

In [5]:
data = []
for company in companies:

    name = company.text.replace(" ", "-").replace(",", "-").replace(".","-").replace("(","-").replace(")","-").replace("|","-").replace("&","-").replace("/","-")
    name = re.sub(r"-+", "-", name).strip("-").lower()

    url = "https://kbis2025.smallworldlabs.com/co/" + name

    r2 = requests.get(url)
    soup2 = BeautifulSoup(r2.text, "html.parser")

    try:
        # Find all divs with class 'profileResponse'
        site = soup2.findAll("div", {"class": "profileResponse"})
        hall, loc, website = "N/A", "N/A", "N/A"  # Initialize defaults

        # Loop through the site divs and match content by patterns
        for div in site:
            text = div.text.strip()
            if "Hall" in text: # Assume text with 'Hall' is the hall location
                hall = text  
            elif div.find('a'):  # Check for website
                website = div.find('a')['href']
            elif "<br/>" in str(div):  # Location field with <br> tag
                loc = div.decode_contents().replace('<br/>', ', ').strip()

        # Split location into City, State, and Country
        city, state, country = "N/A", "N/A", "N/A"
        if loc != "N/A":
            loc_parts = loc.split(', ')
            
            if len(loc_parts) == 3:  # City, State, Country case
                city, state, country = loc_parts[0], loc_parts[1], loc_parts[2]
            elif len(loc_parts) == 2:  # City, Country case
                city, country = loc_parts[0], loc_parts[1]
            elif len(loc_parts) == 1:  # Only City available
                city = loc_parts[0]

        booth = soup2.find("a", {"class" : "generic-option-link font-weight-bold icon-margin-right"})
        booth_info = "N/A"
        if booth:
            # Extract the URL from the href attribute
            booth_url = booth.get("href")

            # Extract the booth information from the text inside the <a> tag
            booth_info = booth.get_text(strip=True).replace("Booth", "").strip()
            booth_info = booth_info.replace("#", "").strip()
            
        # Append data to the list
        data.append({
            'Company Name': company.text,
            'Booth': booth_info,
            'Hall': hall,
            'Website': website,
            'City': city,
            'State': state,
            'Country': country            
        })

    except AttributeError:
        # Handle case where expected elements are not found
        data.append({
            'Company Name': company.text,
            'Booth': "N/A",
            'Hall': "N/A",
            'Website': "N/A",
            'City': "N/A",
            'State': "N/A",
            'Country': "N/A"
        })
        

In [6]:
df = pd.DataFrame(data)
df

,Company Name,Booth,Hall,Website,City,State,Country
0,2020,N2013,North Hall,https://www.cyncly.com,Grand Rapids,MI,United States
1,3CAD,N2013,North Hall,http://www.cyncly.com,Grand Rapids,MI,United States
2,618 Cabinetry Inc.,SL6212,South Hall,https://618cabinetry.com/,Elmhurst,IL,United States
3,A. u. K. Müller GmbH & Co. KG,N/A,N/A,N/A,N/A,N/A,N/A
4,Aalto Marble Inlay,N2070,South Hall,http://www.aaltomarbleinlay.com,Fairfield,IA,United States
...,...,...,...,...,...,...,...
560,"Yukon Group, The",SL1030,South Hall,N/A,Stafford,TX,United States
561,"ZBOM Cabinets Co., Ltd.",SL8015,South Hall,http://www.zbomcabinets.com,Azusa,CA,United States
562,ZDS,W1177,West Hall,http://www.zdsdistribution.com,Phoenix,AZ,United States
563,Zip Water,W1437,West Hall,https://na.zipwater.com,Rosemont,IL,United States


In [7]:
df.to_excel('KBISoutputForDad.xlsx', index=False)